<h2><b>CREDIT CARD APPROVAL PREDICTION PROJECT</b></h2>
<h3><u>Author: Chirag Ingle</u></h3>
<p>It is an automatic credit card approval predictor that uses machine learning techniques.</p>

<h3>NEED OF CREDIT CARD APPROVAL PREDICTION:</h3>

<p>Commercial banks receive a lot of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming.So, this task can be automated with the power of machine learning and most of the banks use the same technique.</p>

<img src='https://cardinsider.com/wp-content/uploads/2023/08/Improve-Your-Chances-For-Credit-Card-Application-Blog-Post.jpg' alt="Image Not Found">

<p>We will use the <i>Credit Card Approval dataset from the UCI Machine Learning Repository.</i> The structure of this notebook is as follows:

<ul><li>We will start off by loading and viewing the dataset.</li>
<li>We will see that the dataset has a mixture of both numerical and non-numerical features, that it contains values from different ranges, plus that it contains a number of missing entries.</li>
<li>We will have to preprocess the dataset to ensure the machine learning model we choose can make good predictions.</li>
<li>After our data is in good shape, we will do some exploratory data analysis to build our intuitions.</li></ul>
Finally, we will build a machine learning model that can predict if an individual's application for a credit card will be accepted.
Before the first step, we are mounting the dataset from the google drive to the google collab.</p>

<h3>Importing Libraries</h3>

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import math
import tensorflow as tf
import warnings
import ydata_profiling
from ydata_profiling import ProfileReport
from IPython.display import HTML
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import xgboost

In [77]:
from sklearn.ensemble import  RandomForestClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.model_selection import train_test_split

In [61]:
#Version Check
tf.__version__

'2.13.0'

In [62]:
sklearn.__version__

'1.2.2'

In [63]:
#ignoring all the warnings
warnings.filterwarnings('ignore')

<h3>Importing Dataset</h3>

In [64]:
data=pd.read_csv("/kaggle/input/credit-card-approval/cc_approvals.data",header=None)

In [65]:
data.sample(5)

0      1       2  3  4  5   6     7  8  9   10 11 12     13    14 15
169  b  37.50   1.125  y  p  d   v  1.50  f  f   0  t  g  00431     0  +
599  b  20.50   2.415  u  g  c   v  2.00  t  t  11  t  g  00200  3000  +
411  b  25.17   3.000  u  g  c   v  1.25  f  t   1  f  g  00000    22  -
388  b  26.67  14.585  u  g  i  bb  0.00  f  f   0  t  g  00178     0  -
163  b  32.00   1.750  y  p  e   h  0.04  t  f   0  t  g  00393     0  +

<h4><b>Inspecting The Applications</b></h4>
<p>The features of this dataset have been anonymized to protect the privacy. The probable features in a typical credit card application are Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus. This gives us a pretty good starting point, and we can map these features with respect to the columns in the output.

The dataset has a mixture of numerical and non-numerical features. This can be fixed with some preprocessing, but before we do that, let's learn about the dataset a bit more to see if there are other dataset issues that need to be fixed.</p>

In [66]:
data.describe()

2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


<h3>3. Handling the missing values (part i)</h3>
<p>We've uncovered some issues that will affect the performance of our machine learning model(s) if they go unchanged:

Our dataset contains both numeric and non-numeric data (specifically data that are of float64, int64 and object types). Specifically, the features 2, 7, 10 and 14 contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values. The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, we can get useful statistical information (like mean, max, and min) about the features that have numerical values. Finally, the dataset has missing values, which we'll take care of in this task. The missing values in the dataset are labeled with '?', which can be seen in the last cell's output.

Now, let's temporarily replace these missing value question marks with NaN.</p>

In [68]:
print(data.tail(17))

data.replace(to_replace='?',value=np.nan,inplace=False)

print(data.tail(17))

    0      1       2  3  4   5   6      7  8  9   10 11 12     13   14 15
673  ?  29.50   2.000  y  p   e   h  2.000  f  f   0  f  g  00256   17  -
674  a  37.33   2.500  u  g   i   h  0.210  f  f   0  f  g  00260  246  -
675  a  41.58   1.040  u  g  aa   v  0.665  f  f   0  f  g  00240  237  -
676  a  30.58  10.665  u  g   q   h  0.085  f  t  12  t  g  00129    3  -
677  b  19.42   7.250  u  g   m   v  0.040  f  t   1  f  g  00100    1  -
678  a  17.92  10.210  u  g  ff  ff  0.000  f  f   0  f  g  00000   50  -
679  a  20.08   1.250  u  g   c   v  0.000  f  f   0  f  g  00000    0  -
680  b  19.50   0.290  u  g   k   v  0.290  f  f   0  f  g  00280  364  -
681  b  27.83   1.000  y  p   d   h  3.000  f  f   0  f  g  00176  537  -
682  b  17.08   3.290  u  g   i   v  0.335  f  f   0  t  g  00140    2  -
683  b  36.42   0.750  y  p   d   v  0.585  f  f   0  f  g  00240    3  -
684  b  40.58   3.290  u  g   m   v  3.500  f  f   0  t  s  00400    0  -
685  b  21.08  10.085  y  p   e   h  1

<h3> 4. Handling the missing values (part ii) </h3>

<p>We replaced all the question marks with NaNs. This is going to help us in the next missing value treatment that we are going to perform.

An important question that gets raised here is why are we giving so much importance to missing values? Can't they be just ignored? Ignoring missing values can affect the performance of a machine learning model heavily. While ignoring the missing values our machine learning model may miss out on information about the dataset that may be useful for its training. Then, there are many models which cannot handle missing values implicitly such as LDA.

So, to avoid this problem, we are going to impute the missing values with a strategy called mean imputation.</p>

In [69]:
# Fill NaN values with mean for numeric columns
numeric_columns = data.select_dtypes(include=np.number).columns
data[numeric_columns] = data[numeric_columns].apply(lambda x: x.fillna(x.mean()))

<h3>5. Handling the missing values (part iii)</h3>
<p>We have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this why the mean imputation strategy would not work here. This needs a different treatment.

We are going to impute these missing values with the most frequent values as present in the respective columns. This is good practice when it comes to imputing missing values for categorical data in general.</p>

In [70]:
# Iterate over each column
for col in data.columns:
    # Check if the column is of object type
    if data[col].dtype == 'object':
        # Impute with the most frequent value
        data[col] = data[col].fillna(data[col].value_counts().index[0])

# If there are still any remaining NaN values, you can fill them with a constant or additional strategies
data = data.fillna(0)  # Filling remaining NaN values with 0, adjust as needed

# Count the number of NaNs in the dataset and print the counts to verify
print(data.isna().sum())


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


In [71]:
data

0      1       2  3  4   5   6     7  8  9   10 11 12     13   14 15
0    b  30.83   0.000  u  g   w   v  1.25  t  t   1  f  g  00202    0  +
1    a  58.67   4.460  u  g   q   h  3.04  t  t   6  f  g  00043  560  +
2    a  24.50   0.500  u  g   q   h  1.50  t  f   0  f  g  00280  824  +
3    b  27.83   1.540  u  g   w   v  3.75  t  t   5  t  g  00100    3  +
4    b  20.17   5.625  u  g   w   v  1.71  t  f   0  f  s  00120    0  +
..  ..    ...     ... .. ..  ..  ..   ... .. ..  .. .. ..    ...  ... ..
685  b  21.08  10.085  y  p   e   h  1.25  f  f   0  f  g  00260    0  -
686  a  22.67   0.750  u  g   c   v  2.00  f  t   2  t  g  00200  394  -
687  a  25.25  13.500  y  p  ff  ff  2.00  f  t   1  t  g  00200    1  -
688  b  17.92   0.205  u  g  aa   v  0.04  f  f   0  f  g  00280  750  -
689  b  35.00   3.375  u  g   c   h  8.29  f  f   0  t  g  00000    0  -

[690 rows x 16 columns]

In [72]:
numeric_data

2     7   10   14
0     0.000  1.25   1    0
1     4.460  3.04   6  560
2     0.500  1.50   0  824
3     1.540  3.75   5    3
4     5.625  1.71   0    0
..      ...   ...  ..  ...
685  10.085  1.25   0    0
686   0.750  2.00   2  394
687  13.500  2.00   1    1
688   0.205  0.04   0  750
689   3.375  8.29   0    0

[690 rows x 4 columns]

<h3>Exploratory Data Analysis</h3>

In [73]:
profile=ProfileReport(data,explorative=True)
profile.to_file('output.html',)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
HTML('output.html')

<h3>6. Preprocessing the data (part i)</h3>
<p>The missing values are now successfully handled.

There is still some minor but essential data preprocessing needed before we proceed towards building our machine learning model. We are going to divide these remaining preprocessing steps into three main tasks:

Convert the non-numeric data into numeric. Split the data into train and test sets. Scale the feature values to a uniform range. First, we will be converting all the non-numeric values into numeric ones. We do this because not only it results in a faster computation but also many machine learning models (like XGBoost) (and especially the ones developed using scikit-learn) require the data to be in a strictly numeric format. We will do this by using a technique called label encoding.</p>

In [75]:
le=LabelEncoder()
# Iterate over all the values of each column and extract their dtypes
for col in data.columns.values:
    # Compare if the dtype is object
    if data[col].dtypes =='object':
    # Use LabelEncoder to do the numeric transformation
        data[col]=le.fit_transform(data[col])

<h3>7. Splitting the dataset into train and test sets</h3>
We have successfully converted all the non-numeric values to numeric ones.

Now, we will split our data into train set and test set to prepare our data for two different phases of machine learning modeling: training and testing. Ideally, no information from the test data should be used to scale the training data or should be used to direct the training process of a machine learning model. Hence, we first split the data and then apply the scaling.

Also, features like DriversLicense and ZipCode are not as important as the other features in the dataset for predicting credit card approvals. We should drop them to design our machine learning model with the best set of features. In Data Science literature, this is often referred to as feature selection.

In [82]:
# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
data = data.iloc[:,:11].join([data.iloc[:,12],data.iloc[:,14:]])
data = data.values

# Segregate features and labels into separate variables
X,y = data[:,0:12] , data[:,13]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)     

<h3>8. Preprocessing the data (part ii)</h3>
The data is now split into two separate sets - train and test sets respectively. We are only left with one final preprocessing step of scaling before we can fit a machine learning model to the data.

Now, let's try to understand what these scaled values mean in the real world. Let's use CreditScore as an example. The credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a CreditScore of 1 is the highest since we're rescaling all the values to the range of 0-1.

In [84]:
# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

<h3>9.Fitting a logistic regression model to the train set</h3>
Essentially, predicting if a credit card application will be approved or not is a classification task. According to UCI, our dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved.

This gives us a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.

Which model should we pick? A question to ask is: are the features that affect the credit card approval decision process correlated with each other? Although we can measure correlation, that is outside the scope of this notebook, so we'll rely on our intuition that they indeed are correlated for now. Because of this correlation, we'll take advantage of the fact that generalized linear models perform well in these cases. Let's start our machine learning modeling with a Logistic Regression model (a generalized linear model).

In [111]:
# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train,y_train)

rfc=RandomForestClassifier()
rfc.fit(rescaledX_train,y_train)

gbc=GradientBoostingClassifier()
gbc.fit(rescaledX_train,y_train)

hgbc=HistGradientBoostingClassifier()
hgbc.fit(rescaledX_train,y_train)

xgb=xgboost.XGBClassifier()
xgb.fit(rescaledX_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

<h3>10. Making predictions and evaluating performance</h3>

But how well does our model perform?

We will now evaluate our model on the test set with respect to classification accuracy. But we will also take a look the model's confusion matrix. In the case of predicting credit card applications, it is equally important to see if our machine learning model is able to predict the approval status of the applications as denied that originally got denied. If our model is not performing well in this aspect, then it might end up approving the application that should have been approved. The confusion matrix helps us to view our model's performance from these aspects.

In [113]:
# Use logreg to predict instances from the test set and store it
y_pred_log = logreg.predict(rescaledX_test)
y_pred_rfc = rfc.predict(rescaledX_test)
y_pred_gbc = gbc.predict(rescaledX_test)
y_pred_hgbc = hgbc.predict(rescaledX_test)
y_pred_xgb = xgb.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of Random Forest Classifier: ", sklearn.metrics.accuracy_score(y_test,y_pred_rfc))
print("Accuracy of HistGradient Boosted Classifier: ", sklearn.metrics.accuracy_score(y_test,y_pred_hgbc))
print("Accuracy of Logistic Regression Classifier: ", sklearn.metrics.accuracy_score(y_test,y_pred_log))
print("Accuracy of Gradient Boosted Classifier: ", sklearn.metrics.accuracy_score(y_test,y_pred_gbc))
print("Accuracy of XG Boost Classifier: ", sklearn.metrics.accuracy_score(y_test,y_pred_xgb))

Accuracy of Random Forest Classifier:  0.8640350877192983
Accuracy of HistGradient Boosted Classifier:  0.8552631578947368
Accuracy of Logistic Regression Classifier:  0.8333333333333334
Accuracy of Gradient Boosted Classifier:  0.7017543859649122
Accuracy of XG Boost Classifier:  0.7324561403508771
[[92 11]
 [27 98]]
[[ 91  12]
 [ 19 106]]


In [114]:
#best output accuracy is given by random forest classifier
print(confusion_matrix(y_test,y_pred_rfc))

[[ 91  12]
 [ 19 106]]


Our model was pretty good! It was able to yield an accuracy score of almost 84%.

For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.

While building this credit card predictor, we tackled some of the most widely-known preprocessing steps such as scaling, label encoding, and missing value imputation. We finished with some machine learning to predict if a person's application for a credit card would get approved or not given some information about that person.